In [2]:
import os
import glob
import pydicom
import cv2
import numpy as np
import shutil

In [3]:
def get_LUT_value(data, window, level):
    try:
        window = window[0]
        #window = 4096
    except TypeError:
        pass
    try:
        level = level[0]
        #level = 2048
    except TypeError:
        pass

    return np.piecewise(data,
                        [data <= (level - 0.5 - (window - 1) / 2),
                         data > (level - 0.5 + (window - 1) / 2)],
                        [0, 255, lambda data: ((data - (level - 0.5)) /
                         (window - 1) + 0.5) * (255 - 0)])


In [4]:
dcm_path = 'D:/Distal_radius/exp3/raw_data' # dcm 파일 폴더
file_extention = '.png' # 파일 확장자
file_dcm_path = [] # jpg 저장할 환자 폴더
list_dcm = os.listdir(dcm_path) # 환자 폴더
list_dcm.sort()
dcm_path = glob.glob(dcm_path + '/*.dcm')

len(dcm_path), len(list_dcm)

(3, 16)

In [5]:
# 저장
print('*' * 30)
print('save start')
print('*' * 30)

file_extention = '.png' 
img_path = 'D:/Distal_radius/exp3/raw_data/png/resize'  #resize 전
save_path = 'D:/Distal_radius/exp3/raw_data/png'
error_path='D:/Distal_radius/exp3/raw_data/png/error'

os.mkdir(save_path)
os.mkdir(img_path)
os.mkdir(error_path)

#new_file_path = os.listdir(dir_path) # jpg 저장할 폴더
#print(new_file_path)
error = []
dimension1, dimension2 = [], []
#new_file_paths = dir_path + new_file_path[i] + '\\'
print(len(dcm_path))

#os.mkdir(save_path)
#os.mkdir(img_path)



for i in range(len(dcm_path)):
    print(dcm_path[i])
    dcm=dcm_path[i]
    ds = pydicom.read_file(dcm, force=True)
    
    window_set = 4096
    level_set = 2048
    
    window_set2 = 1000
    level_set2 = 500
    
    try:
        if ds[(0x0028,0x0004)].value == 'MONOCHROME1':
            try:
                tmp=ds[(0x7FE0,0X0010)].value 
                leng=len(tmp)-(ds.Rows*ds.Columns*2)
                if not leng == 0:
                    tmp=tmp[:-leng]
                    #ds.PixelData=tmp
                    ds[(0x7FE0,0X0010)].value=tmp
                Arraydicom = ds.pixel_array
                dimension = Arraydicom.shape
                x, y = dimension
                if x > y:
                    new_size = x
                    add_size = x-y
                    add_image = np.zeros((new_size, add_size))
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    ArrayDicom = get_LUT_value(ArrayDicom, ds.WindowWidth, ds.WindowCenter)
#                     ArrayDicom = np.hstack((225-ArrayDicom, add_image))
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), 225-ArrayDicom)
                    image = cv2.resize(ArrayDicom, (1024,1024))

                elif y > x:
                    new_size = y
                    add_size = y-x

                    add_image = np.zeros((add_size, new_size))
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, ds.WindowWidth, ds.WindowCenter)
                    
#                     ArrayDicom = np.vstack((225-ArrayDicom, add_image))
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), 255-ArrayDicom)
                    image = cv2.resize(ArrayDicom, (1024,1024))

                else:
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(225-ArrayDicom, ds.WindowWidth, ds.WindowCenter)
                    
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), 225-ArrayDicom)
                    image = cv2.resize(225-ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    cv2.imwrite(os.path.join(save_path, file_name), image)



            except:

                tmp=ds[(0x7FE0,0X0010)].value 
                leng=len(tmp)-(ds.Rows*ds.Columns*2)
                if not leng == 0:
                    tmp=tmp[:-leng]
                    #ds.PixelData=tmp
                    ds[(0x7FE0,0X0010)].value=tmp

                Arraydicom = ds.pixel_array 
                dimension = Arraydicom.shape
                x, y = dimension
                if x > y:
                    new_size = x
                    add_size = x-y

                    add_image = np.zeros((new_size, add_size))
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, window_set, level_set)
                    
#                     ArrayDicom = np.hstack((225-ArrayDicom, add_image))
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), 225-ArrayDicom)
                    image = cv2.resize(225-ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    
                    if pixel_avg < 100:
                        cv2.imwrite(os.path.join(save_path, file_name), image)
                    else:

                        dcm=dcm_path[i]
                        ds = pydicom.read_file(dcm, force=True)
                        tmp=ds[(0x7FE0,0X0010)].value 
                        leng=len(tmp)-(ds.Rows*ds.Columns*2)
                        if not leng == 0:
                            tmp=tmp[:-leng]
                            #ds.PixelData=tmp
                            ds[(0x7FE0,0X0010)].value=tmp
                        Arraydicom = ds.pixel_array
                        dimension = Arraydicom.shape
                        x, y = dimension
                        new_size = x
                        add_size = x-y

                        add_image = np.zeros((new_size, add_size))
                        ds.RescaleIntercept = 0
                        ds.RescaleSlope = 1
                        ArrayDicom = ds.pixel_array
                        ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                        pixel_avg = np.mean(Arraydicom)
                        print(pixel_avg)
                        ArrayDicom = get_LUT_value(ArrayDicom, window_set2, level_set2)

#                         ArrayDicom = np.hstack((225-ArrayDicom, add_image))
                        file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                        print(add_image.shape, ArrayDicom.shape)
                        cv2.imwrite(os.path.join(img_path, file_name), 225-ArrayDicom)
                        image = cv2.resize(225-ArrayDicom, (1024,1024))
                        cv2.imwrite(os.path.join(save_path, file_name), image)

                elif y > x:
                    new_size = y
                    add_size = y-x

                    add_image = np.zeros((add_size, new_size))
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, window_set, level_set)
                    
#                     ArrayDicom = np.vstack((225-ArrayDicom, add_image))
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), 225-ArrayDicom)
                    image = cv2.resize(225-ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    
                    if pixel_avg < 100:
                        cv2.imwrite(os.path.join(save_path, file_name), image)
                    else:
                        dcm=dcm_path[i]
                        ds = pydicom.read_file(dcm, force=True)
                        tmp=ds[(0x7FE0,0X0010)].value 
                        leng=len(tmp)-(ds.Rows*ds.Columns*2)
                        if not leng == 0:
                            tmp=tmp[:-leng]
                            #ds.PixelData=tmp
                            ds[(0x7FE0,0X0010)].value=tmp
                        Arraydicom = ds.pixel_array
                        dimension = Arraydicom.shape
                        x, y = dimension
                        new_size = y
                        add_size = y-x

                        add_image = np.zeros((add_size, new_size))
                        ds.RescaleIntercept = 0
                        ds.RescaleSlope = 1
                        ArrayDicom = ds.pixel_array
                        ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
#                         pixel_avg = np.mean(Arraydicom)
                        print(pixel_avg)
                        ArrayDicom = get_LUT_value(ArrayDicom, window_set2, level_set2)

#                         ArrayDicom = np.vstack((225-ArrayDicom, add_image))
                        file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                        print(add_image.shape, ArrayDicom.shape)
                        cv2.imwrite(os.path.join(img_path, file_name), 225-ArrayDicom)
                        image = cv2.resize(225-ArrayDicom, (1024,1024))
                        cv2.imwrite(os.path.join(save_path, file_name), image)
                        
                else:
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, window_set, level_set)
                    
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), 225-ArrayDicom)
                    image = cv2.resize(225-ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    
                    if pixel_avg < 100:
                        cv2.imwrite(os.path.join(save_path, file_name), image)
                    else:
                        dcm=dcm_path[i]
                        ds = pydicom.read_file(dcm, force=True)
                        tmp=ds[(0x7FE0,0X0010)].value 
                        leng=len(tmp)-(ds.Rows*ds.Columns*2)
                        if not leng == 0:
                            tmp=tmp[:-leng]
                            #ds.PixelData=tmp
                            ds[(0x7FE0,0X0010)].value=tmp
                        Arraydicom = ds.pixel_array
                        dimension = Arraydicom.shape
                        x, y = dimension

                        ds.RescaleIntercept = 0
                        ds.RescaleSlope = 1
                        ArrayDicom = ds.pixel_array
                        ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                        pixel_avg = np.mean(Arraydicom)
                        print(pixel_avg)
                        ArrayDicom = get_LUT_value(ArrayDicom, window_set2, level_set2)

                        file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                        print(add_image.shape, ArrayDicom.shape)
                        cv2.imwrite(os.path.join(img_path, file_name), 225-ArrayDicom)
                        image = cv2.resize(225-ArrayDicom, (1024,1024))
                        cv2.imwrite(os.path.join(save_path, file_name), image)

        elif ds[(0x0028,0x0004)].value == 'MONOCHROME2':
            try:
                tmp=ds[(0x7FE0,0X0010)].value 
                leng=len(tmp)-(ds.Rows*ds.Columns*2)
                if not leng == 0:
                    tmp=tmp[:-leng]
                    #ds.PixelData=tmp
                    ds[(0x7FE0,0X0010)].value=tmp

                Arraydicom = ds.pixel_array 
                dimension = Arraydicom.shape
                x, y = dimension
                if x > y:
                    new_size = x
                    add_size = x-y

                    add_image = np.zeros((new_size, add_size))
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, ds.WindowWidth, ds.WindowCenter)
                    
#                     ArrayDicom = np.hstack((ArrayDicom, add_image))
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), ArrayDicom)
                    image = cv2.resize(ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    cv2.imwrite(os.path.join(save_path, file_name), image)

                elif y > x:
                    new_size = y
                    add_size = y-x

                    add_image = np.zeros((add_size, new_size))
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, ds.WindowWidth, ds.WindowCenter)
                    
#                     ArrayDicom = np.vstack((ArrayDicom, add_image))
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), ArrayDicom)
                    image = cv2.resize(ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    cv2.imwrite(os.path.join(save_path, file_name), image)

                else:
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, ds.WindowWidth, ds.WindowCenter)
                    
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), ArrayDicom)
                    image = cv2.resize(ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    cv2.imwrite(os.path.join(save_path, file_name), image)



            except:

                tmp=ds[(0x7FE0,0X0010)].value 
                leng=len(tmp)-(ds.Rows*ds.Columns*2)
                if not leng == 0:
                    tmp=tmp[:-leng]
                    #ds.PixelData=tmp
                    ds[(0x7FE0,0X0010)].value=tmp

                Arraydicom = ds.pixel_array 
                dimension = Arraydicom.shape
                x, y = dimension
                if x > y:
                    new_size = x
                    add_size = x-y

                    add_image = np.zeros((new_size, add_size))
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, window_set, level_set)
                    
#                     ArrayDicom = np.hstack((ArrayDicom, add_image))
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), ArrayDicom)
                    image = cv2.resize(ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    
                    if pixel_avg < 100:
                        cv2.imwrite(os.path.join(save_path, file_name), image)
                        
                    else:
                        dcm = dcm_path[i]
                        ds = pydicom.read_file(dcm, force=True)
                        tmp=ds[(0x7FE0,0X0010)].value 
                        leng=len(tmp)-(ds.Rows*ds.Columns*2)
                        if not leng == 0:
                            tmp=tmp[:-leng]
                            #ds.PixelData=tmp
                            ds[(0x7FE0,0X0010)].value=tmp
                        Arraydicom = ds.pixel_array
                        dimension = Arraydicom.shape
                        x, y = dimension
                        new_size = x
                        add_size = x-y

                        add_image = np.zeros((new_size, add_size))
                        ds.RescaleIntercept = 0
                        ds.RescaleSlope = 1
                        ArrayDicom = ds.pixel_array
                        ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                        pixel_avg = np.mean(Arraydicom)
                        print(pixel_avg)
                        ArrayDicom = get_LUT_value(ArrayDicom, window_set2, level_set2)

#                         ArrayDicom = np.hstack((ArrayDicom, add_image))
                        file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                        print(add_image.shape, ArrayDicom.shape)
                        cv2.imwrite(os.path.join(img_path, file_name), ArrayDicom)
                        image = cv2.resize(ArrayDicom, (1024,1024))
                        cv2.imwrite(os.path.join(save_path, file_name), image)

                elif y > x:
                    new_size = y
                    add_size = y-x

                    add_image = np.zeros((add_size, new_size))
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, window_set, level_set)
                    
#                     ArrayDicom = np.vstack((ArrayDicom, add_image))
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), ArrayDicom)
                    image = cv2.resize(ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    
                    if pixel_avg < 100:
                        cv2.imwrite(os.path.join(save_path, file_name), image)
                        
                    else:
                        dcm = dcm_path[i]
                        ds = pydicom.read_file(dcm, force=True)
                        tmp=ds[(0x7FE0,0X0010)].value 
                        leng=len(tmp)-(ds.Rows*ds.Columns*2)
                        if not leng == 0:
                            tmp=tmp[:-leng]
                            #ds.PixelData=tmp
                            ds[(0x7FE0,0X0010)].value=tmp
                        Arraydicom = ds.pixel_array
                        dimension = Arraydicom.shape
                        x, y = dimension
                        new_size = x
                        add_size = x-y

                        add_image = np.zeros((add_size, new_size))
                        ds.RescaleIntercept = 0
                        ds.RescaleSlope = 1
                        ArrayDicom = ds.pixel_array
                        ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                        pixel_avg = np.mean(Arraydicom)
                        print(pixel_avg)
                        ArrayDicom = get_LUT_value(ArrayDicom, window_set2, level_set2)

#                         ArrayDicom = np.vstack((ArrayDicom, add_image))
                        file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                        print(add_image.shape, ArrayDicom.shape)
                        cv2.imwrite(os.path.join(img_path, file_name), ArrayDicom)
                        image = cv2.resize(ArrayDicom, (1024,1024))
                        cv2.imwrite(os.path.join(save_path, file_name), image)

                else:
                    ds.RescaleIntercept = 0
                    ds.RescaleSlope = 1
                    ArrayDicom = ds.pixel_array
                    ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                    pixel_avg = np.mean(Arraydicom)
                    print(pixel_avg)
                    ArrayDicom = get_LUT_value(ArrayDicom, window_set, level_set)
                    
                    file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                    print(add_image.shape, ArrayDicom.shape)
                    cv2.imwrite(os.path.join(img_path, file_name), ArrayDicom)
                    image = cv2.resize(ArrayDicom, (1024,1024))
                    pixel_avg = np.mean(image)
                    print(pixel_avg)
                    
                    if pixel_avg < 100:
                        cv2.imwrite(os.path.join(save_path, file_name), image)
                        
                    else:

                        dcm = dcm_path[i]
                        ds = pydicom.read_file(dcm, force=True)
                        tmp=ds[(0x7FE0,0X0010)].value 
                        leng=len(tmp)-(ds.Rows*ds.Columns*2)
                        if not leng == 0:
                            tmp=tmp[:-leng]
                            #ds.PixelData=tmp
                            ds[(0x7FE0,0X0010)].value=tmp
                        Arraydicom = ds.pixel_array
                        dimension = Arraydicom.shape
                        
                        ds.RescaleIntercept = 0
                        ds.RescaleSlope = 1
                        ArrayDicom = ds.pixel_array
                        ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
                        ArrayDicom = get_LUT_value(ArrayDicom, window_set2, level_set2)
                        
                        file_name = dcm.split("\\")[-1].split(".")[0] + file_extention
                        print(add_image.shape, ArrayDicom.shape)
                        cv2.imwrite(os.path.join(img_path, file_name), ArrayDicom)
                        image = cv2.resize(ArrayDicom, (1024,1024))
                        cv2.imwrite(os.path.join(save_path, file_name), image)

        else:
            error.append(dcm)
            shutil.move(dcm, error_path)
    except:
        error.append(dcm)
        shutil.move(dcm, error_path)

print('*' * 30)
print('save done')
print('*' * 30)

******************************
save start
******************************
3
D:/Distal_radius/exp3/raw_data\1306738_X1L.dcm
497.0010080205151
(188, 1580) (1392, 1580)
30.95139842610664
D:/Distal_radius/exp3/raw_data\1968044 X1A.dcm
2897.4598996037776
2897.4598996037776
(340, 2010) (1670, 2010)
44.57523466508331
D:/Distal_radius/exp3/raw_data\1968044 X1L.dcm
2897.4598996037776
2897.4598996037776
(340, 2010) (1670, 2010)
44.57523466508331
******************************
save done
******************************


In [ ]:
error

In [49]:
e_name=os.listdir(error_path)
dcm=pydicom.read_file(error_path+'/'+e_name[0], force=True)

In [50]:
dcm

(0008, 0005) Specific Character Set              CS: 'ISO_IR 6'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'NORMALIZED', 'RT']
(0008, 0016) SOP Class UID                       UI: Computed Radiography Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.392.200036.9125.4.0.4228796102.2515272704.534942696
(0008, 0020) Study Date                          DA: '20021006'
(0008, 0021) Series Date                         DA: '20021006'
(0008, 0022) Acquisition Date                    DA: '20021006'
(0008, 0023) Content Date                        DA: '20021006'
(0008, 0030) Study Time                          TM: '144100.0000'
(0008, 0031) Series Time                         TM: '144100.0000'
(0008, 0032) Acquisition Time                    TM: '144100.0000'
(0008, 0033) Content Time                        TM: '144100.0000'
(0008, 0050) Accession Number                    SH: '215\x00'
(0008, 0060) Modality                            CS: 'C

# 한 영상에서 AP, LAT 영상 분리

In [14]:
import matplotlib.pyplot as plt

In [74]:
path = 'D:/Distal_radius/exp3/raw_data/png/R_AP'
a='D:/Distal_radius/exp3/raw_data/png/resize'
src='D:/Distal_radius/exp3/raw_data/png/resize/R_AP'
png_name = os.listdir(path)

for i in range(len(png_name)):
    shutil.move(a+'/'+png_name[i], src)
    

In [78]:
path='D:/Distal_radius/exp3/raw_data/png/resize/R_AP'
png_name = os.listdir(path)

# i=3
for i in range(len(png_name)):
    img=cv2.imread(path+'/'+png_name[i])
    print(img.shape)
    x_size=int(img.shape[1]/2)
    
    if 'A' in png_name[i]:
        ap_img=img[:,x_size:]
        cv2.imwrite(path+'/AP/'+png_name[i], ap_img)

    elif 'L' in png_name[i]:  
        lat_img=img[:,:x_size]
        cv2.imwrite(path+'/LAT/'+png_name[i], lat_img)

# plt.imshow(ap_img)


(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1191, 1484, 3)
(1191, 1484, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(2010, 1670, 3)
(2010, 1670, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2010, 3)
(1670, 2

AttributeError: 'NoneType' object has no attribute 'shape'